## Image Center Roof Crop Notebook

Authors: Shaurya Tripathi, Shiva Srivastava

In [ ]:
# This files dependencies
from roboflow import Roboflow
import os
import cv2
from PIL import Image
import numpy as np

#### Convert png files to jpeg files

In [ ]:
input_directory = './PNGS'
output_directory = './JPEGS'

os.makedirs(output_directory, exist_ok=True)
files = [f for f in os.listdir(input_directory) if not(os.path.isdir(f))]

for png_file in files:
    png_path = os.path.join(input_directory, png_file)
    image = Image.open(png_path)
    print(png_file)

    jpeg_filename = os.path.splitext(png_file)[0] + '.jpg'
    jpeg_path = os.path.join(output_directory, jpeg_filename)

    image = image.convert("RGB")
    image.save(jpeg_path, "JPEG", quality=100)
    print("Jpeg File saved")

print("Conversion completed.")

In [ ]:
# Check if image is empty
def is_non_empty(img):
   np.reshape(img, (-1,1))
   u, count_unique = np.unique(img, return_counts =True)
   
   return count_unique.size>= 10


#### Setup the Segmentation model

In [ ]:
# Load Segmentation model
## Update version number if new version of model generated
def load_segmentation_model(api_key="9pyHdiM1tENb1DwODp1M", project_name="arkasegmentationproject", version=4):
  from roboflow import Roboflow
  rf = Roboflow(api_key=api_key)
  project = rf.workspace().project(project_name)
  seg_model = project.version(version).model
  return seg_model

In [ ]:
# Return prediction
def outline_pred_and_json(in_path, seg_model):
  return seg_model.predict(in_path, confidence=50).json()

In [ ]:
def cut_bounding_box(img_path, seg_pred_json, out_dir, margin=15):
  image_rgb = cv2.imread(img_path)
  #print(img_path)
  if(seg_pred_json["predictions"] and is_non_empty(image_rgb)):
    for bounding_box in seg_pred_json["predictions"]:
        x0 = bounding_box['x'] - bounding_box['width'] / 2 - margin
        x1 = bounding_box['x'] + bounding_box['width'] / 2 + margin
        y0 = bounding_box['y'] - bounding_box['height'] / 2 - margin
        y1 = bounding_box['y'] + bounding_box['height'] / 2 + margin

        start_point = (int(x0), int(y0))
        end_point = (int(x1), int(y1))

        center_roof_bb = image_rgb[int(y0):int(y1), int(x0):int(x1)]

    os.makedirs(out_dir, exist_ok=True)
    f_name = os.path.basename(img_path)
    out_path = out_dir+f_name
    #print(out_path)
    if(is_non_empty(center_roof_bb)):
        cv2.imwrite(out_path, center_roof_bb)
    #display(Image(filename=out_path))

#### Now get the center roof crop

In [ ]:
seg_model = load_segmentation_model()


In [ ]:
input_directory = './JPEGS'
output_directory = './CROPS/'

os.makedirs(output_directory, exist_ok=True)

jpeg_files = [f for f in os.listdir(input_directory)]
#print(jpeg_files)


for file in jpeg_files:
    img_path = os.path.join(input_directory, file)
    seg_json = outline_pred_and_json(img_path, seg_model)
    cut_bounding_box(img_path, seg_json, output_directory)

print("Center Roof crops done!")